In [5]:
import httpx
import os
from langgraph_sdk import get_client

SUPABASE_URL = os.environ["SUPABASE_URL"]
SUPABASE_ANON_KEY = os.environ["SUPABASE_ANON_KEY"]

email1 = "user1@example.com"
password = "testpass123"
email2 = "user2@example.com"
password2 = "testpass456"

async def login(email: str, password: str):
    """Get an access token for an existing user."""
    async with httpx.AsyncClient() as client:
        response = await client.post(
            f"{SUPABASE_URL}/auth/v1/token?grant_type=password",
            json={
                "email": email,
                "password": password
            },
            headers={
                "apikey": SUPABASE_ANON_KEY,
                "Content-Type": "application/json"
            },
        )
        assert response.status_code == 200
        return response.json()["access_token"]



# Log in as user 1
user1_token = await login(email1, password)
user1_client = get_client(
    url="http://localhost:2024", headers={"Authorization": f"Bearer {user1_token}"}
)

ConnectError: [Errno 8] nodename nor servname provided, or not known

In [ ]:
# Create a thread as user 1
thread = await user1_client.threads.create()
print(f"✅ User 1 created thread: {thread['thread_id']}")

# Try to access without a token
unauthenticated_client = get_client(url="http://localhost:2024")
try:
    await unauthenticated_client.threads.create()
    print("❌ Unauthenticated access should fail!")
except Exception as e:
    print("✅ Unauthenticated access blocked:", e)

# Try to access user 1's thread as user 2
user2_token = await login(email2, password2)
user2_client = get_client(
    url="http://localhost:2024", headers={"Authorization": f"Bearer {user2_token}"}
)

try:
    await user2_client.threads.get(thread["thread_id"])
    print("❌ User 2 shouldn't see User 1's thread!")
except Exception as e:
    print("✅ User 2 blocked from User 1's thread:", e)

In [ ]:
print("🤖 Invoking the assistant with a question...")
input_data = {"messages": [{"role": "human", "content": "hello! what's my most recent repo?"}]}

# Stream the response with cleaner formatting
async for event in user1_client.runs.stream(
    thread_id=thread["thread_id"],
    assistant_id="agent",
    input=input_data,
    stream_mode="updates",
):
    if event.event == "metadata":
        print(f"📋 Run started (ID: {event.data.get('run_id', 'Unknown')[:8]}...)")
        
    elif event.event == "updates":
        # Handle agent updates (AI messages and tool calls)
        if "agent" in event.data:
            for msg in event.data["agent"]["messages"]:
                if msg["type"] == "ai":
                    # Check if AI is making tool calls
                    if msg.get("tool_calls"):
                        for tool_call in msg["tool_calls"]:
                            tool_name = tool_call.get("name", "Unknown")
                            print(f"🔧 Calling tool: {tool_name}")
                            # Show tool arguments in a clean way
                            if "args" in tool_call and tool_call["args"]:
                                args_str = str(tool_call["args"])
                                if len(args_str) > 100:
                                    args_str = args_str[:100] + "..."
                                print(f"   └─ Args: {args_str}")
                    
                    # Show AI response content (if not just tool calls)
                    elif msg.get("content") and msg["content"].strip():
                        print(f"💬 Assistant Response:")
                        # Show full response for final answers
                        content = msg["content"]
                        if len(content) > 500:
                            print(f"{content}\n")
                        else:
                            print(f"{content}\n")
        
        # Handle tool responses
        elif "tools" in event.data:
            for msg in event.data["tools"]["messages"]:
                if msg["type"] == "tool":
                    tool_name = msg.get("name", "Unknown tool")
                    print(f"✅ Tool '{tool_name}' completed")
                    # Show brief summary of tool response
                    content = msg.get("content", "")
                    if content:
                        # For research tool, try to count results
                        try:
                            import json
                            results = json.loads(content)
                            if isinstance(results, list):
                                print(f"   └─ Found {len(results)} results")
                            else:
                                print(f"   └─ Result: {str(content)[:150]}...")
                        except:
                            print(f"   └─ Result: {str(content)[:150]}...")

print("\n" + "="*50)
print("🎉 Assistant conversation completed!")
print("="*50)